# Разработка алгоритма преобразования данных

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## 1. Загрузка данных

Импортируем библиотеки, получим все данные и рассмотрим общую информацию об импортированных данных.

In [1]:
import numpy as np
import pandas as pd
import doctest
from sklearn.metrics import r2_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LinearRegression

In [2]:
try:
    data = pd.read_csv('/datasets/insurance.csv')     
except:
    data = pd.read_csv('c:/Users/User/Downloads/insurance.csv')

In [3]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [5]:
data.shape

(5000, 5)

In [6]:
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']

## 2. Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Предсказания $a$ не изменятся, качество линейной регрессии останется прежним

**Обоснование:**

Для доказательства подставим вместо $X$ - $XP$
$$
a_{new} = XPw_{new}
$$$$
w_{new} = ((XP)^T XP)^{-1}(XP)^T y
$$
подставим $w_{new}$ в $a_{new}$ и преобразуем:
$$
a_{new} = X P ((XP)^T XP)^{-1}(XP)^T y = \\X P (P^TX^T XP)^{-1}P^T X^T y = \\X P (X^TXP)^{-1} (P^T)^{-1} P^T X^T y = \\X P P^{-1}(X^TX)^{-1} (P^T)^{-1} P^T X^T y = \\X E (X^TX)^{-1} E  X^T y = \\X (X^TX)^{-1} X^T y =  X w
$$

**Пояснения:**

- Раскрываем транспонирование по свойству: $(AB)^T = B^T A^T$

- $X^TX$ - квадратная обратимая марица, $P$ - обратимая по опеределнию, далее раскрываем скобки по свойству $(AB)^{-1} = B^{-1} A^{-1}$ 2 раза

- Используя ассоциативность умножения матриц и определение обратной матрицы получим ответ:

$$
a_{new} = a
$$

В рассуждениях одно допущение, что $X^TX$ - обратима, но в общем случае она может оказаться не обратимой.

## 2. Алгоритм преобразования

**Алгоритм**

Для защиты информации на этапе поиска коэффициентов регрессии и предсказаний регрессии будем умножать матрицу признаков на обратимую матрицу $Y$, которая будет генерироваться случайным образом.

**Обоснование**

Матрица $Y$ должна иметь необходимую размерность $(nxn)$, где n - количество признаков для регрессии.
Таким образом матрица $Z$ будет иметь туже размерность, что и матрица $X$. Обратная матрица $Y$ существует только для квадратных невырожденных матриц (определитель которых не равен нулю).

Пример:

$
X = \begin{pmatrix}
1 & 2 \\
2 & 3 \\
4 & 5  
\end{pmatrix}
\qquad 
Y = \begin{pmatrix}
1 & 0 \\
2 & 3 \\  
\end{pmatrix}
\qquad det  Y = 3 $

Найдем значние Z:

$
Z = \begin{pmatrix}
1 & 2 \\
2 & 3 \\
4 & 5  
\end{pmatrix} \begin{pmatrix}
1 & 0 \\
2 & 3 \\  
\end{pmatrix} = \begin{pmatrix}
1*1+2*2 & 1*0+2*3  \\
2*1+3*2 & 2*0+3*3  \\
4*1+5*2 & 4*0+5*3   
\end{pmatrix} = \begin{pmatrix}
5 & 6 \\
8 & 9 \\
14 & 15  
\end{pmatrix}$

После этого добавляем нулевой столбец и вводим данные в линейную регрессию.


In [7]:
X = np.random.randn(4, 2)
Y = np.random.randn(2, 2)

Z = np.linalg.inv(Y)

X_ = X @ Y
X

array([[ 2.76427798,  0.39621704],
       [-0.87594423,  1.46575197],
       [-0.63121325, -0.26439512],
       [ 0.45535811,  0.43094178]])

In [8]:
X_ @ Z

array([[ 2.76427798,  0.39621704],
       [-0.87594423,  1.46575197],
       [-0.63121325, -0.26439512],
       [ 0.45535811,  0.43094178]])

## Проверка алгоритма

Запрограммируйте этот алгоритм, применив матричные операции. Проверьте, что качество линейной регрессии из sklearn не отличается до и после преобразования. Примените метрику R2.

In [9]:
class BlaBla(TransformerMixin, BaseEstimator):
    
    
    def fit(self, features, y=None, random_state=None):
        if random_state:
            np.random.seed(random_state)
            
        features_dim = features.shape[1]
        P = np.random.normal(size=(features_dim, features_dim))
        np.linalg.inv(P)
            
        self.P = P
        self.features = features
        return self
        
           
    def transform(self, features, y=None):  
        return features.dot(self.P)

In [10]:
doctest.testmod()

TestResults(failed=0, attempted=0)

In [11]:
features_new = BlaBla().fit_transform(features, random_state=12345)
features.shape, features_new.shape

((5000, 4), (5000, 4))

In [12]:
linreg = LinearRegression()

linreg.fit(features, target)
predict1 = linreg.predict(features)

linreg.fit(features_new, target)
predict2 = linreg.predict(features_new)

In [13]:
r2_score(target, predict1)

0.42494550286668

In [14]:
r2_score(target, predict2)

0.42494550286667954

In [15]:
np.isclose(r2_score(target, predict1), r2_score(target, predict2))

True

#### Вывод:
Метрика R2 осталась без особых изменений, даже после преобразования. 
Следовательно, при умножении признаков на обратимую матрицу не повлияет на качество линейной регрессию, как мы ранее и доказали. Наше доказательство / предположение оказались верными. 